شبکه کانولوشنال عمیقی برای پیش‌بینی کلاس‌ها در دیتاست CIFAR10 بسازید. 15 درصد دیتاست را برای تست در نظر بگیرید. عملکرد این شبکه را با بهترین شبکه فصل قبل مقایسه کنید.

تعداد لایه‌ها و تعداد و ابعاد کرنل‌های در نظر گرفته شده را گزارش کنید.
مدت زمان آموزش شبکه را گزارش کرده و با شبکه Dense فصل قبل مقایسه کنید و accuracy پیش‌بینی را روی داده‌های تست گزارش کنید.
عملکرد دو برنامه زمانی Exponential و OneCycle را برای 50 epoch مقایسه کنید.
تفاوت حضور و عدم حضور Pooling layer روی شبکه را برای 50 epoch بررسی کنید. مدت زمان آموزش را گزارش کنید و accuracy و loss را نمایش دهید.
روی لایه Pooling اثر strideهای 2و 4 را برای 50 epoch بررسی کنید و accuracy و loss را نمایش دهید.
همین مسئله را با یکی از شبکه‌های قدرتمندی که در این فصل دیدید حل کنید. یک بار تمام شبکه را فریز کرده و بار دیگر یک لایه از آن را trainable کنید. این مقایسه را برای 50 epoch انجام دهید و accuracy و loss هر دو حالت را در یک گراف نمایش دهید.
* قسمت کد پاسخ خود را به صورت لینک Colab یا فایل نوت‌بوک با پسوند .ipynb بفرستید. برای تسریع فرایند تصحیح کد خود را کامنت‌گذاری کنید.

* حتما قبل از ارسال کد را اجرا کنید تا نتایج در فایل ارسالی وجود داشته باشد.

* قسمت تشریحی پاسخ خود را در یک فایل با فرمت ورد یا پی‌دی‌اف ارسال کنید.

1. تقسیم‌بندی داده‌ها:

ابتدا 85 درصد از داده‌ها را برای آموزش و 15 درصد را برای تست جدا می‌کنیم.

In [1]:
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# بارگذاری دیتاست CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# تبدیل برچسب‌ها به one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# تقسیم داده‌های آموزش به دو بخش: 85% برای آموزش و 15% برای تست
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.15, random_state=42)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 19s 0us/step


2. ساخت شبکه کانولوشنال عمیق (CNN):

در این بخش، شبکه CNN با تعدادی لایه کانولوشن و Pooling تعریف می‌شود.

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout

# ساخت شبکه کانولوشنال عمیق
def create_cnn_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Dropout(0.25),

        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Dropout(0.25),

        Conv2D(256, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Dropout(0.25),

        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    return model


3. آموزش و مقایسه با شبکه Dense:

برای آموزش شبکه، از optimizer و loss مناسب استفاده کرده و مدل را روی داده‌های آموزش و اعتبارسنجی به مدت 50 epoch اجرا می‌کنیم.

In [3]:
from tensorflow.keras.optimizers import Adam

# ساخت مدل CNN
cnn_model = create_cnn_model()

# کامپایل مدل
cnn_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# آموزش مدل
history_cnn = cnn_model.fit(x_train, y_train, epochs=50, validation_data=(x_val, y_val), batch_size=64, verbose=1)

# ارزیابی مدل روی داده‌های تست
test_loss, test_accuracy = cnn_model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
665/665 ━━━━━━━━━━━━━━━━━━━━ 23s 18ms/step - accuracy: 0.3482 - loss: 2.0188 - val_accuracy: 0.4535 - val_loss: 1.7326
Epoch 2/50
665/665 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.5431 - loss: 1.2800 - val_accuracy: 0.6289 - val_loss: 1.0408
Epoch 3/50
665/665 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.6197 - loss: 1.0719 - val_accuracy: 0.6085 - val_loss: 1.1979
Epoch 4/50
665/665 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.6718 - loss: 0.9322 - val_accuracy: 0.7069 - val_loss: 0.8267
Epoch 5/50
665/665 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.7065 - loss: 0.8374 - val_accuracy: 0.6920 - val_loss: 0.9298
Epoch 6/50
665/665 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.7269 - loss: 0.7869 - val_accuracy: 0.6577 - val_loss: 1.0861
Epoch 7/50
665/665 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.7317 - loss: 0.7732 - val_accuracy: 0.7484 - val_loss: 0.7459
Epoch 8/50
665/665 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.7568 - loss: 0.6944 - val_accuracy:

4. استفاده از برنامه‌های زمانی Exponential و OneCycle:

برای مقایسه این دو برنامه زمانی (learning rate schedules)، ابتدا مدل‌ها را با استفاده از هر کدام از این روش‌ها آموزش داده و سپس مقایسه می‌کنیم.

Exponential Decay:

In [4]:
from tensorflow.keras.callbacks import LearningRateScheduler

# تابع برنامه زمانی Exponential Decay
def exponential_decay(epoch, lr):
    return lr * 0.96 ** epoch

# آموزش مدل با Exponential Decay
exp_decay_schedule = LearningRateScheduler(exponential_decay)
history_exp_decay = cnn_model.fit(x_train, y_train, epochs=50, validation_data=(x_val, y_val), callbacks=[exp_decay_schedule], batch_size=64, verbose=1)


Epoch 1/50
665/665 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9219 - loss: 0.2294 - val_accuracy: 0.8383 - val_loss: 0.6492 - learning_rate: 0.0010
Epoch 2/50
665/665 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9220 - loss: 0.2206 - val_accuracy: 0.8213 - val_loss: 0.7019 - learning_rate: 9.6000e-04
Epoch 3/50
665/665 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.9264 - loss: 0.2183 - val_accuracy: 0.8303 - val_loss: 0.6525 - learning_rate: 8.8474e-04
Epoch 4/50
665/665 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9270 - loss: 0.2123 - val_accuracy: 0.8389 - val_loss: 0.6397 - learning_rate: 7.8276e-04
Epoch 5/50
665/665 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9294 - loss: 0.2036 - val_accuracy: 0.8485 - val_loss: 0.6177 - learning_rate: 6.6483e-04
Epoch 6/50
665/665 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9362 - loss: 0.1835 - val_accuracy: 0.8457 - val_loss: 0.6143 - learning_rate: 5.4209e-04
Epoch 7/50
665/665 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9

OneCycle:

In [5]:
from tensorflow.keras.optimizers.schedules import OneCycleLR

# برنامه زمانی OneCycle
one_cycle_schedule = OneCycleLR(max_lr=0.01, total_steps=50*len(x_train)//64)
optimizer_onecycle = Adam(learning_rate=one_cycle_schedule)

cnn_model.compile(optimizer=optimizer_onecycle, loss='categorical_crossentropy', metrics=['accuracy'])
history_onecycle = cnn_model.fit(x_train, y_train, epochs=50, validation_data=(x_val, y_val), batch_size=64, verbose=1)


ImportError: cannot import name 'OneCycleLR' from 'tensorflow.keras.optimizers.schedules' (/usr/local/lib/python3.10/dist-packages/keras/_tf_keras/keras/optimizers/schedules/__init__.py)

5. اثر Pooling و بررسی Strideها:

برای بررسی اثر لایه‌های Pooling و همچنین مقایسه strideهای مختلف، دو مدل با و بدون لایه‌های Pooling آموزش می‌دهیم و سپس نتایج را مقایسه می‌کنیم.

مدل بدون لایه Pooling:

In [ ]:
def create_cnn_without_pooling():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        Dropout(0.25),

        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        Dropout(0.25),

        Conv2D(256, (3, 3), activation='relu'),
        BatchNormalization(),
        Dropout(0.25),

        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    return model


مدل با Strideهای مختلف:

In [ ]:
def create_cnn_with_stride(stride_value):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2), strides=stride_value),
        Dropout(0.25),

        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2), strides=stride_value),
        Dropout(0.25),

        Conv2D(256, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2), strides=stride_value),
        Dropout(0.25),

        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    return model


6. مقایسه و نمایش نتایج:

برای مقایسه عملکرد مدل‌ها، دقت و زیان آن‌ها را برای هر نوع تنظیمات روی داده‌های تست رسم می‌کنیم.

In [ ]:
import matplotlib.pyplot as plt

def plot_history(histories, key='accuracy'):
    plt.figure(figsize=(10,8))

    for name, history in histories:
        plt.plot(history.history[key], label=name)

    plt.title(f'Model {key}')
    plt.xlabel('Epochs')
    plt.ylabel(key)
    plt.legend()
    plt.show()

# رسم گراف‌های مقایسه‌ای دقت و زیان
plot_history([('CNN', history_cnn), ('Exponential Decay', history_exp_decay), ('OneCycle', history_onecycle)], key='val_accuracy')
plot_history([('CNN', history_cnn), ('Exponential Decay', history_exp_decay), ('OneCycle', history_onecycle)], key='val_loss')


7. نتیجه‌گیری:

مدت زمان آموزش: زمان آموزش شبکه CNN باید با شبکه Dense مقایسه شود. معمولاً شبکه‌های کانولوشنال زمان بیشتری برای آموزش نیاز دارند، اما دقت بالاتری را روی داده‌های تصویر به دست می‌آورند.

دقت پیش‌بینی: دقت پیش‌بینی شبکه CNN با لایه‌های کانولوشن به طور معمول بهتر از شبکه Dense خواهد بود، چرا که شبکه‌های کانولوشنال به خوبی می‌توانند ویژگی‌های مکانی داده‌های تصویری را استخراج کنند.

اثر Pooling و Stride: لایه‌های Pooling و همچنین مقدار stride در Pooling می‌تواند به کاهش زمان محاسباتی و بهبود دقت مدل کمک کند.